In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import pickle
import numpy as np

#TO BE FIXED: not very clean, but revealed to be necessary on my computer... (SD)
import sys
sys.path.append("/Users/doncieux/git/diversity_algorithms_dev")
sys.path.append("/Users/doncieux/git/fastsim_gym")

from diversity_algorithms.experiments.maze_ns_exploration import *


# Analyzing an experiment

This notebook contains the skeleton of code toanalyze the result of a run.

The experiment needs to have been generated with experiments/maze_ns_exploration.py. Exemple:
```
python3 -m scoop maze_ns_exploration.py -p 200 -g 1000 -e 0
```
to call novelty search for 1000 generations and a population size of 200. The run is parallelized with scoop. The '-e 0' argument avoids to test the evolvability per individual (that takes ages as it has to generate X new individuals per individual...).

Once the run is finished, a directory contains all the files generated during th run (depends on the options you have transmitted to the script).  containing the population of the last generation, the logbook and the name of the dir where the dump files are, is generated.



In [ ]:
res_dir="<my res file>" # put here the name of the automatically generated run name (something like 2019_08_13-16:39:05_0)

params=np.load(res_dir+"/params.npz", allow_pickle=True)
info=open(res_dir+"/info.log","r")
print("Infos on the run: "+" ".join(info.readlines()[1:]))
info.close()
print("Run params: ")
for k in params.keys():
    print("   %s: "%(k)+str(params[k]))

  
nb_bin=int(params["NB_BIN"])
min_x=params["MIN_X"]
max_x=params["MAX_X"]

print("Loading logbook for the last generation, i.e. gen %d"%(params["NGEN"]))
logbook=np.load(res_dir+"/logbook_gen%d.npz"%(params["NGEN"]), allow_pickle=True)
print("Keys in the logbook: "+str(list(logbook.keys())))


## Plotting Behavior descriptors

If you have dumped behavior descriptors, here is the code to plot them.

### All generated behavior descriptors

In [ ]:
# loading points
import importlib
import diversity_algorithms.analysis.data_utils
import diversity_algorithms.analysis.maze_plot

bdfiles = diversity_algorithms.analysis.data_utils.get_bdfiles_per_gen([res_dir])
points = diversity_algorithms.analysis.data_utils.get_points_per_gen_from_bdfiles(bdfiles)
allpoints = diversity_algorithms.analysis.data_utils.merge_gens(points)

# To plot only the bd from gen x to gen y:
#points_x_y = diversity_algorithms.analysis.data_utils.merge_gens(points,min_gen=x,max_gen=y)


print("Read BD: len allpoints=%d"%(len(allpoints)))


In [ ]:
# plotting bd
grid_all=build_grid(min_x, max_x, nb_bin)
update_grid(grid_all,min_x, max_x, allpoints)
print("Coverage for all points: %f  nb_bin=%d"%(coverage(grid_all),nb_bin))

diversity_algorithms.analysis.maze_plot.plot_points(allpoints,title="Behavior descriptors")

### Only the behavior descriptors of the population (i.e. selected individuals, from parents+offspring in general)

In [ ]:
# loading the data
bdpopfiles = diversity_algorithms.analysis.data_utils.get_bdpopfiles_per_gen([res_dir])
pointspop = diversity_algorithms.analysis.data_utils.get_points_per_gen_from_bdfiles(bdpopfiles)
allpointspop = diversity_algorithms.analysis.data_utils.merge_gens(pointspop)

print("Read BD: len allpointspop=%d"%(len(allpointspop)))


In [ ]:
# plotting bd
grid_all_pop=build_grid(min_x, max_x, nb_bin)
update_grid(grid_all_pop,min_x, max_x, allpointspop)
print("Coverage for all points pop: %f  nb_bin=%d"%(coverage(grid_all_pop),nb_bin))

diversity_algorithms.analysis.maze_plot.plot_points(allpointspop,title="Behavior descriptors")

## Looking at stats


In [ ]:
print("Keys in the logbook: "+str(list(logbook.keys())))
print(str(type(logbook["population_glob_cov"])))

In [ ]:
#print(logbook["glob_cov"])
gcovpop=[gc[0] for gc in logbook["population_glob_cov"]]
gdistpop=[gc[1] for gc in logbook["population_glob_cov"]]
gcovoff=[gc[0] for gc in logbook["offspring_glob_cov"]]
gdistoff=[gc[1] for gc in logbook["offspring_glob_cov"]]


print("Global coverage")
fig, axs = plt.subplots(1, 2, figsize=(15, 5))
fig.suptitle("Values of set of indivs on a sliding window of %d generations"%(params["GLOBAL_WINDOW_SIZE"]))
axs[0].set_ylim(0,1)
axs[0].plot(logbook["gen"],gcovpop, label="population (selected indivs)")
axs[0].plot(logbook["gen"],gcovoff, label="offspring (all generated indivs)")
axs[1].plot(logbook["gen"],gdistpop, label="population (selected indivs)")
axs[1].plot(logbook["gen"],gdistoff, label="offspring (all generated indivs)")
axs[0].legend(loc='upper left')
axs[1].legend(loc='upper left')
axs[0].set_title("Coverage")
axs[1].set_title("Distance to uniform")

plt.show()

